# Introducción a la API de tf.keras

# Introducción a Keras LSTM para series de tiempo multivariadas

## Autor

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 

## References

1. [Introducción a Redes LSTM](Intro_LSTM.ipynb)
2. [Time Series Forecasting with LSTMs using TensorFlow 2 and Keras in Python](https://towardsdatascience.com/time-series-forecasting-with-lstms-using-tensorflow-2-and-keras-in-python-6ceee9c6c651/)
3. [Tensoflow-Time series forecasting](https://www.tensorflow.org/tutorials/structured_data/time_series)

# Introducción


Este cuaderno es  una introducción a la predicción de series de tiempo utilizando redes neuronales recurrentes (RNN). Esto se cubre en dos partes: primero, pronosticará una serie de tiempo univariada, luego pronosticará una serie de tiempo multivariada.


# Importar la librerias requeridas

In [ ]:
import tensorflow as tf

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

# El conjunto de datos weather 


Usaremos el conjunto de datos (dataset) [weather time series dataset recorded by the Max Planck Institute for Biogeochemistry.]()

Este dataset contiene 14 variables (features) diferentes tales como temperatura ambiente, presión atmosférica, y humedad. Estos datos fueron recolectados cada 10 minutos, comenzando en 2003. Por eficiencia, usaremos los datos recolectados entre 2009 y 2016. Estos datos fueron preparados por François Chollet para su libro Deep Learning with Python. En total son 420551 registros de datos.

In [ ]:
# download teh data and store it in your system
zip_path = tf.keras.utils.get_file(
    origin='https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip',
    fname='jena_climate_2009_2016.csv.zip',
    extract=True)
# after download the file is in this path
csv_path, _ = os.path.splitext(zip_path) # /home/alvaro/.keras/datasets/jena_climate_2009_2016.csv'

In [ ]:
df = pd.read_csv(csv_path)

## Una primera mirada a los datos 

In [ ]:
df.head()

In [ ]:
df.describe().T


Como puede verse arriba, se registra una observación cada 10 minutos. Esto significa que, durante una hora, se tendrán 6 observaciones. Del mismo modo, en un  día se tienen 144 (6x24) observaciones. En total son 420551 observaciones.

Dado un tiempo específico, digamos que desea predecir la temperatura 6 horas en el futuro. Para hacer esta predicción, se elige usar 5 días de observaciones. Por lo tanto, crearía una ventana que contiene las últimas 720 (5x144) observaciones para entrenar el modelo. Son posibles muchas de estas configuraciones, lo que hace que este conjunto de datos sea bueno para experimentar.

La siguiente función devuelve las ventanas de tiempo descritas anteriormente para entrenar el modelo. El parámetro *history_size* es el tamaño de la ventana de información pasada. El *target_size* es qué tan lejos en el futuro necesita el modelo aprender a predecir; *target_size* es la etiqueta que debe predecirse.

In [ ]:
def univariate_data(dataset, start_index, end_index, history_size, target_size):
  data = []
  labels = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index):
    indices = range(i-history_size, i)
    # Reshape data from (history_size,) to (history_size, 1)
    data.append(np.reshape(dataset[indices], (history_size, 1)))
    labels.append(dataset[i+target_size])
  return np.array(data), np.array(labels)

En las partes  siguientes, las primeras 300,000 filas de datos serán el conjunto de datos de entrenamiento, y el resto será el conjunto de datos de validación (120551). Esto equivale a unos 2100 días de datos de entrenamiento.


In [ ]:
TRAIN_SPLIT = 300000

Se coloca una semilla para gerantizar reproductibidad

In [ ]:
tf.random.set_seed(13)

# Parte 1: Pronóstico de una serie de tiempo univariada


Primero, se entrenará un modelo usando solo una característica (temperatura), y lo luego se usará para hacer predicciones de ese valor en el futuro.

Primero extraigamos solamente la temperatura del conjunto de datos.

In [ ]:
uni_data = df['T (degC)']
uni_data.index = df['Date Time']
uni_data.columns = ['T (degC)']
uni_data.head()

Observemos como lucen los datos a lo largo del tiempo.

In [ ]:
uni_data.plot(subplots=True)

In [ ]:
uni_data = uni_data.values

## Estandarización (Rescalado) de los datos

Es importante escalar las características antes de entrenar una red neuronal. La estandarización es una forma común de hacer esta escala restando la media y dividiendo por la desviación estándar de cada característica. También podría usar un método *tf.keras.utils.normalize* que reescala los valores para tener para tener media 0 y varianza 1.

In [ ]:
uni_train_mean = uni_data[:TRAIN_SPLIT].mean()
uni_train_std = uni_data[:TRAIN_SPLIT].std()

uni_data = (uni_data-uni_train_mean)/uni_train_std


Ahora creamos los datos para el modelo univariado. Para la parte 1, el modelo recibirá las últimas 20 observaciones de temperatura registradas, y necesita aprender a predecir la temperatura en el siguiente paso.

In [ ]:
# def univariate_data(dataset, start_index, end_index, history_size, target_size):

In [ ]:
univariate_past_history = 20
univariate_future_target = 0

x_train_uni, y_train_uni = univariate_data(uni_data, 0, TRAIN_SPLIT,
                                           univariate_past_history,
                                           univariate_future_target)
x_val_uni, y_val_uni = univariate_data(uni_data, TRAIN_SPLIT, None,
                                       univariate_past_history,
                                       univariate_future_target)

Esto es lo que la función *univariate_data* retorna.

In [ ]:
print ('Ventana  de la historia pasada')
print (x_train_uni[1])
print ('\n Temperature objetivo (target) para predecir')
print (y_train_uni[1])

Ahora que se han creado los datos, echemos un vistazo a un solo ejemplo. La información dada a la red se da en azul y debe predecir el valor en la cruz roja.

In [ ]:
def create_time_steps(length):
  return list(range(-length, 0))

In [ ]:
def show_plot(plot_data, delta, title):
  labels = ['Historia', 'Futuro Verdadero' , 'Predicción del Modelo']
  marker = ['.-', 'rx', 'go']
  time_steps = create_time_steps(plot_data[0].shape[0])
  if delta:
    future = delta
  else:
    future = 0

  plt.title(title)
  for i, x in enumerate(plot_data):
    if i:
      plt.plot(future, plot_data[i], marker[i], markersize=10,
               label=labels[i])
    else:
      plt.plot(time_steps, plot_data[i].flatten(), marker[i], label=labels[i])
  plt.legend()
  plt.xlim([time_steps[0], (future+5)*2])
  plt.xlabel('Salto de tiempo (Time-Step)')
  return plt

In [ ]:
show_plot([x_train_uni[0], y_train_uni[0]], 0, 'Ejemplo de Muestra')
plt.show()

## Modelo línea base: Baseline


Antes de proceder a entrenar un modelo, primero establezcamos una línea base simple. Dado un punto de entrada, el método de línea de base analiza todo el historial y predice que el siguiente punto será el promedio de las últimas 20 observaciones.

In [ ]:
def baseline(history):
  return np.mean(history)

In [ ]:
show_plot([x_train_uni[0], y_train_uni[0], baseline(x_train_uni[0])], 0,
           'Ejemplo de Predicción con el modelo Línea Base')
plt.show()

# Redes  Neuronales Recurrentes


Una red neuronal recurrente (RNR) es un tipo de red neuronal muy adecuada para datos de series temporales. Los RNR procesan una serie temporal paso a paso, manteniendo un estado interno que resume la información que han visto hasta ahora.  En este cuaderno, se utilizará una capa RNR especializada llamada Memoria a largo plazo (LSTM). Para más detalles, lea el cuaderno [Introducción a redes LSTM.](Intro_LSTM.ipynb)

Ahora usemos *tf.data.Dataset* para mezclar, agrupar y almacenar en caché el conjunto de datos.

In [ ]:
BATCH_SIZE = 256
BUFFER_SIZE = 10000

train_univariate = tf.data.Dataset.from_tensor_slices((x_train_uni, y_train_uni))
train_univariate = train_univariate.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_univariate = tf.data.Dataset.from_tensor_slices((x_val_uni, y_val_uni))
val_univariate = val_univariate.batch(BATCH_SIZE).repeat()

## Breve revisión de cache, shuffle, batch y repeat

### ¿Cómo trabaja dataset.shuffle()?

### ¿Cómo trabaja dataset.repeat() ?

### ¿ Qué produce dataset.batch?

### ¿ Qué es el caché?

Los datos almacenados en caché son archivos, scripts, imágenes y otros elementos multimedia almacenados en su dispositivo después de abrir una aplicación o visitar un sitio web por primera vez. Estos datos se utilizan para cargar rápidamente información sobre la aplicación o el sitio web cada vez que se revisa. 

La memoria caché tiene una latencia extremadamente baja, lo que significa que se puede acceder muy rápidamente. La otra cara de la baja latencia significa que no se puede almacenar mucha memoria. Esta es la razón por la cual los archivos de pequeño tamaño  se almacenan en la memoria caché. 

<figure>
<center>
<img src="./Imagenes/time_series.png" width="600" height="400" align="center"/>
</center>
<figcaption>
<p style="text-align:center">Diagrama Representando los datos luego de organizarlos por lotes (batching)"</p>
</figcaption>
</figure>

Como cualquier red implementada en *tf.keras*  la capa de entrada requiere require el tamaño de entrada de los datos.


In [ ]:
#simple_lstm_model = tf.keras.models.Sequential([
#    tf.keras.layers.LSTM(8, input_shape=x_train_uni.shape[-2:]),
#    tf.keras.layers.Dense(1)
#])

simple_lstm_model = tf.keras.models.Sequential()
simple_lstm_model.add(tf.keras.layers.LSTM(8, input_shape=x_train_uni.shape[-2:]))
simple_lstm_model.add(tf.keras.layers.Dense(1))

## Compila el modelo

In [ ]:
simple_lstm_model.compile(optimizer='adam', loss='mae')

Hagamos la predicción de un muestra para chequear la salida del modelo.

In [ ]:
for x, y in val_univariate.take(1):
    print(simple_lstm_model.predict(x).shape)

## Entrenamiento del modelo línea Base

Vamos a entrenar el modelo ahora. Debido al gran tamaño del conjunto de datos, en aras de ahorrar tiempo, cada época (epoch) solo se ejecutará durante 200 pasos, en lugar de los datos de entrenamiento completos como se hace normalmente.


In [ ]:
EVALUATION_INTERVAL = 200
EPOCHS = 10

history = simple_lstm_model.fit(train_univariate, epochs=EPOCHS,
                      steps_per_epoch=EVALUATION_INTERVAL,
                      validation_data=val_univariate, validation_steps=50)

### Evaluación

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend();

## Predicción usando el modelo LSTM simple

Ahora que se ha entrenado su LSTM simple, intentemos hacer algunas predicciones.

In [ ]:
for x, y in val_univariate.take(3):
  plot = show_plot([x[0].numpy(), y[0].numpy(),
                    simple_lstm_model.predict(x)[0]], 0, ' Modelo LSTM Simple')
  plot.show()

Esto se ve mejor que la línea de base. Ahora que ha visto los conceptos básicos, pasemos a la segunda parte, donde trabajará con una serie de tiempo multivariante.

# Parte 2: Pronóstico de una serie de tiempo multivariada


El conjunto de datos original contiene catorce variables (features). Para simplificar, esta sección considera solo tres de los catorce originales. Las características utilizadas son la temperatura del aire, la presión atmosférica y la densidad del aire.

Para usar más funciones, agregue sus nombres a esta lista.

In [ ]:
features_considered = ['p (mbar)', 'T (degC)', 'rho (g/m**3)']

In [ ]:
features = df[features_considered]
features.index = df['Date Time']
features.head()


Echemos un vistazo a cómo cada una de estas características varía con el tiempo.


In [ ]:
features.plot(subplots=True)
plot.show()

Como se mencionó, el primer paso será estandarizar el conjunto de datos utilizando la media y la desviación estándar de los datos de entrenamiento.

In [ ]:
dataset = features.values
data_mean = dataset[:TRAIN_SPLIT].mean(axis=0)
data_std = dataset[:TRAIN_SPLIT].std(axis=0)

In [ ]:
dataset = (dataset-data_mean)/data_std

## Modelo de un Paso


En una configuración de un  paso, el modelo aprende a predecir un solo punto en el futuro en función del historial proporcionado.

La función a continuación realiza la misma tarea de ventanas que antes, sin embargo, aquí muestra la observación pasada en función del tamaño de paso dado.

In [ ]:
def multivariate_data(dataset, target, start_index, end_index, history_size,
                      target_size, step, single_step=False):
  data = []
  labels = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index):
    indices = range(i-history_size, i, step)
    data.append(dataset[indices])

    if single_step:
      labels.append(target[i+target_size])
    else:
      labels.append(target[i:i+target_size])

  return np.array(data), np.array(labels)

En este cuaderno, la red muestra datos de los últimos cinco (5) días, es decir, 720 observaciones que se muestrean cada hora. El muestreo se realiza cada  hora, ya que no se espera un cambio drástico en 60 minutos. Por lo tanto, 120 observaciones representan la historia de los últimos cinco días. Para el modelo de predicción de un solo paso, la etiqueta para un punto de datos es la temperatura dentro de 12 horas. Para crear una etiqueta para esto, se utiliza la temperatura después de 72 (12 * 6) observaciones.

In [ ]:
past_history = 720
future_target = 72
STEP = 6

x_train_single, y_train_single = multivariate_data(dataset, dataset[:, 1], 0,
                                                   TRAIN_SPLIT, past_history,
                                                   future_target, STEP,
                                                   single_step=True)
x_val_single, y_val_single = multivariate_data(dataset, dataset[:, 1],
                                               TRAIN_SPLIT, None, past_history,
                                               future_target, STEP,
                                               single_step=True)

Veamos un puntos de datos de un paso.

In [ ]:
print ('Single window of past history : {}'.format(x_train_single[0].shape))

In [ ]:
train_data_single = tf.data.Dataset.from_tensor_slices((x_train_single, y_train_single))
train_data_single = train_data_single.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_data_single = tf.data.Dataset.from_tensor_slices((x_val_single, y_val_single))
val_data_single = val_data_single.batch(BATCH_SIZE).repeat()

In [ ]:
single_step_model = tf.keras.models.Sequential()
single_step_model.add(tf.keras.layers.LSTM(32,
                                           input_shape=x_train_single.shape[-2:]))
single_step_model.add(tf.keras.layers.Dense(1))

## Compila

In [ ]:
single_step_model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='mae')

Revisemoa una predicción simple

In [ ]:
for x, y in val_data_single.take(1):
  print(single_step_model.predict(x).shape)

## Entrena el modelo

In [ ]:
single_step_history = single_step_model.fit(train_data_single, epochs=EPOCHS,
                                            steps_per_epoch=EVALUATION_INTERVAL,
                                            validation_data=val_data_single,
                                            validation_steps=50)

In [ ]:
def plot_train_history(history, title):
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs = range(len(loss))

  plt.figure()

  plt.plot(epochs, loss, 'b', label='Training loss')
  plt.plot(epochs, val_loss, 'r', label='Validation loss')
  plt.title(title)
  plt.legend()

  plt.show()

In [ ]:
plot_train_history(single_step_history,
                   'Entrnamiento del Modelo de un paso y función de pérdida')

## Predecir un valor futuro de un solo paso


Ahora que el modelo está entrenado, hagamos algunas predicciones de muestra. El modelo tiene el historial de tres features en los últimos cinco días muestreados cada hora (120 puntos de datos), ya que el objetivo es predecir la temperatura, el gráfico solo muestra la temperatura pasada. La predicción se hace un día en el futuro (de ahí la brecha entre la historia y la predicción).

In [ ]:
for x, y in val_data_single.take(3):
  plot = show_plot([x[0][:, 1].numpy(), y[0].numpy(),
                    single_step_model.predict(x)[0]], 12,
                   'Predicciones de un paso')
  plot.show()

## Modelo Multipaso


En un modelo de predicción de varios pasos, dado un historial pasado, el modelo necesita aprender a predecir un rango de valores futuros. Por lo tanto, a diferencia de un modelo de un solo paso, donde solo se predice un único punto futuro, un modelo de varios pasos predice una secuencia del futuro.

Para el modelo de varios pasos, los datos de entrenamiento nuevamente consisten en grabaciones de los últimos cinco días muestreados cada hora. Sin embargo, aquí, el modelo necesita aprender a predecir la temperatura durante las próximas 12 horas. Como se toma una anulación cada 10 minutos, el resultado es 72 predicciones. Para esta tarea, el conjunto de datos debe prepararse en consecuencia, por lo tanto, el primer paso es crearlo nuevamente, pero con una ventana de destino diferente.

In [ ]:
future_target = 72
x_train_multi, y_train_multi = multivariate_data(dataset, dataset[:, 1], 0,
                                                 TRAIN_SPLIT, past_history,
                                                 future_target, STEP)
x_val_multi, y_val_multi = multivariate_data(dataset, dataset[:, 1],
                                             TRAIN_SPLIT, None, past_history,
                                             future_target, STEP)

Revisemos una muestra en un punto.

In [ ]:
print ('Single window of past history : {}'.format(x_train_multi[0].shape))
print ('\n Target temperature to predict : {}'.format(y_train_multi[0].shape))

In [ ]:
train_data_multi = tf.data.Dataset.from_tensor_slices((x_train_multi, y_train_multi))
train_data_multi = train_data_multi.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_data_multi = tf.data.Dataset.from_tensor_slices((x_val_multi, y_val_multi))
val_data_multi = val_data_multi.batch(BATCH_SIZE).repeat()

Plot de una muestra en un punto.

In [ ]:
def multi_step_plot(history, true_future, prediction):
  plt.figure(figsize=(12, 6))
  num_in = create_time_steps(len(history))
  num_out = len(true_future)

  plt.plot(num_in, np.array(history[:, 1]), label='History')
  plt.plot(np.arange(num_out)/STEP, np.array(true_future), 'bo',
           label='True Future')
  if prediction.any():
    plt.plot(np.arange(num_out)/STEP, np.array(prediction), 'ro',
             label='Predicted Future')
  plt.legend(loc='upper left')
  plt.show()

En este gráfico y otros similares, el historial y los datos futuros se muestrean cada hora.

In [ ]:
for x, y in train_data_multi.take(1):
  multi_step_plot(x[0], y[0], np.array([0]))

In [ ]:
for x, y in train_data_multi.take(1):
  multi_step_plot(x[0], y[0], np.array([0]))

Dado que la tarea aquí es un poco más complicada que la tarea anterior, el modelo ahora consta de dos capas LSTM. Finalmente, dado que se hacen 72 predicciones, la capa densa genera 72 predicciones.

In [ ]:
multi_step_model = tf.keras.models.Sequential()
multi_step_model.add(tf.keras.layers.LSTM(32,
                                          return_sequences=True,
                                          input_shape=x_train_multi.shape[-2:]))
multi_step_model.add(tf.keras.layers.LSTM(16, activation='relu'))
multi_step_model.add(tf.keras.layers.Dense(72))

multi_step_model.compile(optimizer=tf.keras.optimizers.RMSprop(clipvalue=1.0), loss='mae')

Veamos como predice el modelo antes de entrenarlo.



In [ ]:
for x, y in val_data_multi.take(1):
  print (multi_step_model.predict(x).shape)


## Entrena el modelo

In [ ]:
multi_step_history = multi_step_model.fit(train_data_multi, epochs=EPOCHS,
                                          steps_per_epoch=EVALUATION_INTERVAL,
                                          validation_data=val_data_multi,
                                          validation_steps=50)

In [ ]:
plot_train_history(multi_step_history, 'Entrenamiento del Modelo Multipaso y función de pérdida')

## Predice un futuro multipaso


Veamos ahora qué tan bien la red ha aprendido a predecir el futuro.

In [ ]:
for x, y in val_data_multi.take(3):
  multi_step_plot(x[0], y[0], multi_step_model.predict(x)[0])

# Los siguientes pasos


Este cuaderno fue una introducción rápida a la predicción de series de tiempo utilizando un RNR. Ahora puede intentar predecir el mercado de valores y convertirse en multimillonario.

Además, también puede escribir un generador para generar datos (en lugar de la función uni / multivariate_data), que sería más eficiente en la memoria. También puedes ver esto [time series windowing](https://www.tensorflow.org/guide/data#time_series_windowing). Úselo para mejorar el código de este cuaderno.